In [1]:
import numpy as np
import cv2

import PIL.Image as Image
import os
import pathlib
import pandas
import csv
import keras

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split

In [2]:
img_folder = r"\data"

df = pandas.read_csv("CoordinateList.csv")
tups = []
for row in df[["NAME", "Label"]].iterrows():
    name = row[1]["NAME"]
    label = row[1]["Label"]
    file_path = os.path.join(img_folder, name)
    for file in os.listdir(file_path):
        # open image and save in the object
        Str = file_path + str("\\") + file
        img_object = cv2.imread(Str)
        img_object_64 = cv2.resize(img_object, (128, 128))
        img_object = np.array(img_object_64)/255
        tups.append((file, img_object_64, img_object, label))
#         print(img_object.shape)

tups[0]
# for monument in df[["NAME", "Label"]]:
#     x = os.listdir(os.path.join(img_folder, monument))
#     print(monument)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '.. \\docs\\img\\Taj Mahal'

In [ ]:
X = []
Y = []
names = []
for name, img, x_, y_ in tups:
    X.append((name, img, x_))
    Y.append(float(y_))
X_train_orig, X_test_orig, Y_train, Y_test = train_test_split(X, Y, random_state=0, train_size=0.9)
X_train_names = [_[0] for _ in X_train_orig]
X_train_images = [_[1] for _ in X_train_orig]
X_train = np.array([_[2] for _ in X_train_orig])
X_test_names = [_[0] for _ in X_test_orig]
X_test_images = [_[1] for _ in X_test_orig]
X_test = np.array([_[2] for _ in X_test_orig])
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
print(X_train.shape)
print(Y_train.shape)
#X_train = np.array(X_train_orig[1])
#X_test = np.array(X_test_orig[1])
#X_train
#X = np.array(X)
#Y = np.array(Y)

#print("Train Size", len(X_train))
#print("Test Size", len(X_test))
#X_train.shape

In [ ]:
cnn = models.Sequential([
    layers.Conv2D(filters=33, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(filters=25, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(filters=17, kernel_size=(5, 5), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(filters=9, kernel_size=(5, 5), activation='relu'),
    layers.Conv2D(filters=3, kernel_size=(5, 5), activation='relu'),
    layers.Flatten(),
    layers.Dense(15, activation='relu'), 
    layers.Dense(7, activation='relu'),
    layers.Dense(3, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
metrics = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'), 
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]
cnn.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=metrics)
cnn.summary()

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
cnn.fit(X_train, Y_train, epochs=100)

In [ ]:
cnn.evaluate(X_test, Y_test)

In [ ]:
def plot_sample(x, y, name):
    plt.figure(figsize = (15,2))
    plt.imshow(x)
    plt.xlabel("Predict: " + ("\"Monument\" " if y >= 0.5 else "\"Random\" ") + "Actual: \""+ str(name) +"\"")

In [ ]:
Y_pred = cnn(X_test)
actuals = [1 if pred >= 0.5 else 0 for pred in Y_test]
preds = [1 if pred >= 0.5 else 0 for pred in Y_pred]
print("Total tests:", len(preds))
print("All positives:", sum(preds))

print("Correct Predictions: ")
for name, img, pred, actual in zip(X_test_names, X_test_images, preds, actuals):
    if pred == actual:
        plot_sample(img, pred, name)

In [ ]:
print("Wrong Predictions: ")
for name, img, pred, actual in zip(X_test_names, X_test_images, preds, actuals):
    if pred != actual:
        plot_sample(img, pred, name)